## On this Notebook, I am going to access to Meteo web from Catalonia's goverment, in order to download authomatically all DATA available from Tarragona (Complex Educatiu) codi_estacio 'XE', Meteo station selected, since the beginning (2009). 

# 1. Download DATA

In [ ]:
#Install Sodapy to download data from Meteorological web
! pip install sodapy

In [ ]:
#Import libraries required
import os
import pandas as pd
import numpy as np
from sodapy import Socrata
import datetime
import operator

In [ ]:
socrata_domain = "analisi.transparenciacatalunya.cat"
socrata_dataset_identifier = "nzvn-apee"
#socrata_token = os.environ.get("None")

client = Socrata(socrata_domain, None)
print(
    "Domain: {domain:}\nSession: {session:}\nURI Prefix: {uri_prefix:}".format(
        **client.__dict__
    )
)

#metadata = client.get_metadata(socrata_dataset_identifier)
#[x["name"] for x in metadata["columns"]]

results = client.get(socrata_dataset_identifier,
                    limit=4000000,
                    codi_estacio="XE")
    
df = pd.DataFrame.from_dict(results)
df

Domain: analisi.transparenciacatalunya.cat
Session: <requests.sessions.Session object at 0x7f42f6b6c690>
URI Prefix: https://


,id,codi_estacio,codi_variable,data_lectura,valor_lectura,codi_estat,codi_base,data_extrem
0,XE360502091530,XE,36,2009-02-05T15:30:00.000,197,V,SH,NaN
1,XE340502091530,XE,34,2009-02-05T15:30:00.000,996,V,SH,NaN
2,XE350502091530,XE,35,2009-02-05T15:30:00.000,0,V,SH,NaN
3,XE440502091600,XE,44,2009-02-05T16:00:00.000,60,V,SH,2009-02-05T16:05:00.000
4,XE420502091600,XE,42,2009-02-05T16:00:00.000,14.4,V,SH,2009-02-05T16:28:00.000
...,...,...,...,...,...,...,...,...
3447813,XE721407210700,XE,72,2021-07-14T07:00:00.000,0,NaN,SH,2021-07-14T07:00:00.000
3447814,XE721407210730,XE,72,2021-07-14T07:30:00.000,0,NaN,SH,2021-07-14T07:30:00.000
3447815,XE721407210800,XE,72,2021-07-14T08:00:00.000,0,NaN,SH,2021-07-14T08:00:00.000
3447816,XE721407210830,XE,72,2021-07-14T08:30:00.000,0,NaN,SH,2021-07-14T08:30:00.000


In [ ]:
#Data object should be modified to datetime, to later on be merged with Pollutant data.
df['data_lectura'] = pd.to_datetime(df['data_lectura'], dayfirst=True)

In [ ]:
#Let's check dataframe size
df.shape

(3447818, 8)

In [ ]:
#Let's check type features
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3447818 entries, 0 to 3447817
Data columns (total 8 columns):
 #   Column         Dtype         
---  ------         -----         
 0   id             object        
 1   codi_estacio   object        
 2   codi_variable  object        
 3   data_lectura   datetime64[ns]
 4   valor_lectura  object        
 5   codi_estat     object        
 6   codi_base      object        
 7   data_extrem    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 210.4+ MB


In [ ]:
#Date is sorted by ascending value
df1 = df.sort_values('data_lectura',ascending=True)
df1

,id,codi_estacio,codi_variable,data_lectura,valor_lectura,codi_estat,codi_base,data_extrem
0,XE360502091530,XE,36,2009-02-05 15:30:00,197,V,SH,NaN
1,XE340502091530,XE,34,2009-02-05 15:30:00,996,V,SH,NaN
2,XE350502091530,XE,35,2009-02-05 15:30:00,0,V,SH,NaN
12,XE500502091600,XE,50,2009-02-05 16:00:00,2.7,V,SH,2009-02-05T16:25:00.000
11,XE510502091600,XE,51,2009-02-05 16:00:00,130,V,SH,NaN
...,...,...,...,...,...,...,...,...
3447719,XE501407210930,XE,50,2021-07-14 09:30:00,7.3,NaN,SH,2021-07-14T09:40:00.000
3447720,XE511407210930,XE,51,2021-07-14 09:30:00,304,NaN,SH,NaN
3447721,XE721407210930,XE,72,2021-07-14 09:30:00,0,NaN,SH,2021-07-14T09:30:00.000
3447713,XE341407210930,XE,34,2021-07-14 09:30:00,1018.3,NaN,SH,NaN


# 2. Select time period to be analyzed

In [ ]:
#Desired period to analyze is 2010-2020. Period decision is made because of many values from 2021 have not been confirmed yet.
#In that case, samples are got by 30 minutes time 
df2 = df1[(df1['data_lectura'] >= '2010-01-01 00:00:01') & (df1['data_lectura'] <= '2020-12-31 23:59:59')]
df2

,id,codi_estacio,codi_variable,data_lectura,valor_lectura,codi_estat,codi_base,data_extrem
218529,XE420101100030,XE,42,2010-01-01 00:30:00,11,V,SH,2010-01-01T00:54:00.000
218528,XE300101100030,XE,30,2010-01-01 00:30:00,7.6,V,SH,NaN
218527,XE330101100030,XE,33,2010-01-01 00:30:00,42,V,SH,NaN
218526,XE340101100030,XE,34,2010-01-01 00:30:00,996,V,SH,NaN
218525,XE400101100030,XE,40,2010-01-01 00:30:00,11.2,V,SH,2010-01-01T00:31:00.000
...,...,...,...,...,...,...,...,...
3412020,XE403112202330,XE,40,2020-12-31 23:30:00,10.3,V,SH,2020-12-31T23:33:00.000
3412021,XE423112202330,XE,42,2020-12-31 23:30:00,9.9,V,SH,2020-12-31T23:52:00.000
3412012,XE033112202330,XE,3,2020-12-31 23:30:00,60,V,SH,2020-12-31T23:56:00.000
3412023,XE503112202330,XE,50,2020-12-31 23:30:00,5.6,V,SH,2020-12-31T23:37:00.000


In [ ]:
#Let's check new dataframe size
df2.shape

(3080024, 8)

In [ ]:
#Let's reduce dataframe to columns desired: date, feature and value
df3=df2[['data_lectura','codi_variable','valor_lectura']]
df3

,data_lectura,codi_variable,valor_lectura
218529,2010-01-01 00:30:00,42,11
218528,2010-01-01 00:30:00,30,7.6
218527,2010-01-01 00:30:00,33,42
218526,2010-01-01 00:30:00,34,996
218525,2010-01-01 00:30:00,40,11.2
...,...,...,...
3412020,2020-12-31 23:30:00,40,10.3
3412021,2020-12-31 23:30:00,42,9.9
3412012,2020-12-31 23:30:00,3,60
3412023,2020-12-31 23:30:00,50,5.6


# 3. Structuring Dataset

In [ ]:
#Let's generate a pivot table to allocate meteorological features in columns as features and date samples as rows
table = df3.pivot_table(index='data_lectura', columns='codi_variable', aggfunc=np.sum,)
table

valor_lectura                        ...                        
codi_variable                   1       2   3   30   31  ...    42  44    50   51 72
data_lectura                                             ...                        
2010-01-01 00:30:00           997     996  43  7.6  274  ...    11  41  13.8  250  0
2010-01-01 01:00:00           997     996  44  8.1  282  ...  10.6  41  15.8  297  0
2010-01-01 01:30:00           997     996  48  6.8  276  ...  10.2  44  13.8  295  0
2010-01-01 02:00:00           997     997  48  8.4  284  ...   9.7  45  15.1  293  0
2010-01-01 02:30:00           998     997  48    8  280  ...   9.5  45  15.6  294  0
...                           ...     ...  ..  ...  ...  ...   ...  ..   ...  ... ..
2020-12-31 21:30:00        1007.6  1007.4  66  1.2  325  ...   8.1  61   2.1  324  0
2020-12-31 22:00:00        1007.5  1007.2  68  0.8  280  ...   7.9  65     2  280  0
2020-12-31 22:30:00        1007.3  1006.9  73  0.4   42  ...   7.2  69   1.1  343  0
2020-12-31 23:00:00        1006.9  1006.7  74  1.3  267  ...     7  58     5  262  0
2020-12-31 23:30:00        1006.8  1006.5  60  2.7  277  ...   9.9  57   5.6  284  0

[192652 rows x 16 columns]

In [ ]:
#Reduce a unique column row
table = pd.DataFrame(table.to_records())
table

,data_lectura,"('valor_lectura', '1')","('valor_lectura', '2')","('valor_lectura', '3')","('valor_lectura', '30')","('valor_lectura', '31')","('valor_lectura', '32')","('valor_lectura', '33')","('valor_lectura', '34')","('valor_lectura', '35')","('valor_lectura', '36')","('valor_lectura', '40')","('valor_lectura', '42')","('valor_lectura', '44')","('valor_lectura', '50')","('valor_lectura', '51')","('valor_lectura', '72')"
0,2010-01-01 00:30:00,997,996,43,7.6,274,11.1,42,996,0,0,11.2,11,41,13.8,250,0
1,2010-01-01 01:00:00,997,996,44,8.1,282,10.8,42,997,0,0,11,10.6,41,15.8,297,0
2,2010-01-01 01:30:00,997,996,48,6.8,276,10.5,45,997,0,0,10.6,10.2,44,13.8,295,0
3,2010-01-01 02:00:00,997,997,48,8.4,284,10,47,997,0,0,10.2,9.7,45,15.1,293,0
4,2010-01-01 02:30:00,998,997,48,8,280,9.5,46,997,0,0,9.7,9.5,45,15.6,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192647,2020-12-31 21:30:00,1007.6,1007.4,66,1.2,325,8.5,64,1007.5,0,0,9.1,8.1,61,2.1,324,0
192648,2020-12-31 22:00:00,1007.5,1007.2,68,0.8,280,8.2,66,1007.3,0,0,8.5,7.9,65,2,280,0
192649,2020-12-31 22:30:00,1007.3,1006.9,73,0.4,42,7.5,71,1007.1,0,0,7.9,7.2,69,1.1,343,0
192650,2020-12-31 23:00:00,1006.9,1006.7,74,1.3,267,7.9,69,1006.8,0,0,10,7,58,5,262,0


In [ ]:
#Date is sorted by ascending value
table = table.sort_values(['data_lectura'],ascending=True)
table

,data_lectura,"('valor_lectura', '1')","('valor_lectura', '2')","('valor_lectura', '3')","('valor_lectura', '30')","('valor_lectura', '31')","('valor_lectura', '32')","('valor_lectura', '33')","('valor_lectura', '34')","('valor_lectura', '35')","('valor_lectura', '36')","('valor_lectura', '40')","('valor_lectura', '42')","('valor_lectura', '44')","('valor_lectura', '50')","('valor_lectura', '51')","('valor_lectura', '72')"
0,2010-01-01 00:30:00,997,996,43,7.6,274,11.1,42,996,0,0,11.2,11,41,13.8,250,0
1,2010-01-01 01:00:00,997,996,44,8.1,282,10.8,42,997,0,0,11,10.6,41,15.8,297,0
2,2010-01-01 01:30:00,997,996,48,6.8,276,10.5,45,997,0,0,10.6,10.2,44,13.8,295,0
3,2010-01-01 02:00:00,997,997,48,8.4,284,10,47,997,0,0,10.2,9.7,45,15.1,293,0
4,2010-01-01 02:30:00,998,997,48,8,280,9.5,46,997,0,0,9.7,9.5,45,15.6,294,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192647,2020-12-31 21:30:00,1007.6,1007.4,66,1.2,325,8.5,64,1007.5,0,0,9.1,8.1,61,2.1,324,0
192648,2020-12-31 22:00:00,1007.5,1007.2,68,0.8,280,8.2,66,1007.3,0,0,8.5,7.9,65,2,280,0
192649,2020-12-31 22:30:00,1007.3,1006.9,73,0.4,42,7.5,71,1007.1,0,0,7.9,7.2,69,1.1,343,0
192650,2020-12-31 23:00:00,1006.9,1006.7,74,1.3,267,7.9,69,1006.8,0,0,10,7,58,5,262,0


In [ ]:
#Show columns
table.columns = ['data_lectura','1','2','3','30','31','32','33','34','35','36','40','42','44','50','51','72']

In [ ]:
#Features values should be transformed to float to be interpreted mathematically
table[['1','2','3','30','31','32','33','34','35','36','40','42','44','50','51','72']] = table[['1','2','3','30','31','32','33','34','35','36','40','42','44','50','51','72']].astype(float)

In [ ]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192652 entries, 0 to 192651
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   data_lectura  192652 non-null  datetime64[ns]
 1   1             192480 non-null  float64       
 2   2             192483 non-null  float64       
 3   3             192418 non-null  float64       
 4   30            192633 non-null  float64       
 5   31            192436 non-null  float64       
 6   32            192413 non-null  float64       
 7   33            192487 non-null  float64       
 8   34            192553 non-null  float64       
 9   35            192652 non-null  float64       
 10  36            192536 non-null  float64       
 11  40            192378 non-null  float64       
 12  42            192379 non-null  float64       
 13  44            192416 non-null  float64       
 14  50            192630 non-null  float64       
 15  51            192

In [ ]:
#Table is grouped by daily measures in order to be consistency with Pollutant dataset. It means, daily samples
table1 = table.groupby(table['data_lectura'].dt.date).mean()
table1

,1,2,3,30,31,32,33,34,35,36,40,42,44,50,51,72
data_lectura,,,,,,,,,,,,,,,,
2010-01-01,1003.531915,1002.680851,45.191489,5.848936,267.234043,11.278723,44.170213,1003.127660,0.000000,71.574468,11.444681,11.102128,43.042553,12.021277,271.957447,0.000000
2010-01-02,1016.875000,1016.312500,57.020833,3.537500,220.875000,9.833333,55.625000,1016.645833,0.000000,61.458333,10.056250,9.604167,54.312500,6.714583,216.187500,0.000000
2010-01-03,1016.458333,1016.125000,79.250000,1.527083,101.083333,9.666667,78.000000,1016.312500,0.000000,93.895833,9.927083,9.418750,76.687500,2.870833,118.208333,0.000000
2010-01-04,1008.625000,1008.354167,82.583333,1.802083,81.458333,9.406250,81.625000,1008.500000,0.250000,36.604167,9.572917,9.233333,80.500000,3.570833,82.333333,0.027083
2010-01-05,1000.083333,999.833333,86.187500,1.868750,158.625000,9.716667,85.458333,999.979167,0.079167,14.375000,9.787500,9.654167,84.625000,3.497917,174.458333,0.022917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-27,1009.404167,1008.766667,65.937500,1.931250,217.395833,6.831250,63.083333,1009.087500,0.000000,59.062500,7.322917,6.295833,60.708333,4.272917,240.083333,0.000000
2020-12-28,995.566667,994.956250,49.062500,5.014583,265.958333,10.668750,47.375000,995.262500,0.000000,106.604167,10.983333,10.360417,45.750000,10.270833,269.041667,0.000000
2020-12-29,1001.879167,1001.518750,47.687500,3.875000,258.208333,10.533333,46.104167,1001.689583,0.000000,91.479167,10.879167,10.147917,44.708333,8.314583,270.229167,0.000000


In [ ]:
table2 = table1.reset_index()
table2

,data_lectura,1,2,3,30,31,32,33,34,35,36,40,42,44,50,51,72
0,2010-01-01,1003.531915,1002.680851,45.191489,5.848936,267.234043,11.278723,44.170213,1003.127660,0.000000,71.574468,11.444681,11.102128,43.042553,12.021277,271.957447,0.000000
1,2010-01-02,1016.875000,1016.312500,57.020833,3.537500,220.875000,9.833333,55.625000,1016.645833,0.000000,61.458333,10.056250,9.604167,54.312500,6.714583,216.187500,0.000000
2,2010-01-03,1016.458333,1016.125000,79.250000,1.527083,101.083333,9.666667,78.000000,1016.312500,0.000000,93.895833,9.927083,9.418750,76.687500,2.870833,118.208333,0.000000
3,2010-01-04,1008.625000,1008.354167,82.583333,1.802083,81.458333,9.406250,81.625000,1008.500000,0.250000,36.604167,9.572917,9.233333,80.500000,3.570833,82.333333,0.027083
4,2010-01-05,1000.083333,999.833333,86.187500,1.868750,158.625000,9.716667,85.458333,999.979167,0.079167,14.375000,9.787500,9.654167,84.625000,3.497917,174.458333,0.022917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,2020-12-27,1009.404167,1008.766667,65.937500,1.931250,217.395833,6.831250,63.083333,1009.087500,0.000000,59.062500,7.322917,6.295833,60.708333,4.272917,240.083333,0.000000
4011,2020-12-28,995.566667,994.956250,49.062500,5.014583,265.958333,10.668750,47.375000,995.262500,0.000000,106.604167,10.983333,10.360417,45.750000,10.270833,269.041667,0.000000
4012,2020-12-29,1001.879167,1001.518750,47.687500,3.875000,258.208333,10.533333,46.104167,1001.689583,0.000000,91.479167,10.879167,10.147917,44.708333,8.314583,270.229167,0.000000
4013,2020-12-30,1010.639583,1010.081250,49.250000,6.022917,295.020833,9.764583,48.312500,1010.368750,0.000000,77.208333,9.947917,9.558333,47.354167,12.089583,293.333333,0.000000


# 4. Import Features label

In [ ]:
socrata_domain = "analisi.transparenciacatalunya.cat"
socrata_dataset_identifier = "4fb2-n3yi"
#socrata_token = os.environ.get("None")

client = Socrata(socrata_domain, None)
print(
    "Domain: {domain:}\nSession: {session:}\nURI Prefix: {uri_prefix:}".format(
        **client.__dict__
    )
)

results = client.get(socrata_dataset_identifier)    
v = pd.DataFrame.from_dict(results)
v.head()

Domain: analisi.transparenciacatalunya.cat
Session: <requests.sessions.Session object at 0x7f424d65b110>
URI Prefix: https://


,codi_variable,nom_variable,unitat,acronim,codi_tipus_var,decimals
0,72,Precipitació màxima en 1 minut,mm,PPTx1min,DAT,1
1,3,Humitat relativa màxima,%,HRx,DAT,0
2,30,Velocitat del vent a 10 m (esc.),m/s,VV10,DAT,1
3,31,Direcció de vent 10 m (m. 1),°,DV10,DAT,0
4,32,Temperatura,°C,T,DAT,1


In [ ]:
#Dataframe creation
v1 = pd.DataFrame(v.to_records())
v1

,index,codi_variable,nom_variable,unitat,acronim,codi_tipus_var,decimals
0,0,72,Precipitació màxima en 1 minut,mm,PPTx1min,DAT,1
1,1,3,Humitat relativa màxima,%,HRx,DAT,0
2,2,30,Velocitat del vent a 10 m (esc.),m/s,VV10,DAT,1
3,3,31,Direcció de vent 10 m (m. 1),°,DV10,DAT,0
4,4,32,Temperatura,°C,T,DAT,1
5,5,33,Humitat relativa,%,HR,DAT,0
6,6,34,Pressió atmosfèrica,hPa,P,DAT,1
7,7,35,Precipitació,mm,PPT,DAT,1
8,8,36,Irradiància solar global,W/m²,RS,DAT,0
9,9,38,Gruix de neu a terra,mm,GNEU,DAT,0


In [ ]:
#Let's create a Dictionary to relabel columns
dict = pd.Series(v1.codi_variable.values,index=v1.nom_variable).to_dict()
dict

{'Direcció de la ratxa màxima del vent a 10 m': '51',
 'Direcció de la ratxa màxima del vent a 2 m': '57',
 'Direcció de la ratxa màxima del vent a 6 m': '54',
 'Direcció de vent 10 m (m. 1)': '31',
 'Direcció del vent a 2 m (m. 1)': '47',
 'Direcció del vent a 6 m (m. 1)': '49',
 'Gruix de neu a terra': '38',
 'Humitat relativa': '33',
 'Humitat relativa màxima': '3',
 'Humitat relativa mínima': '44',
 'Irradiància neta': '59',
 'Irradiància solar global': '36',
 'Precipitació': '35',
 'Precipitació màxima en 1 minut': '72',
 'Pressió atmosfèrica': '34',
 'Pressió atmosfèrica màxima': '1',
 'Pressió atmosfèrica mínima': '2',
 'Ratxa màxima del vent a 10 m': '50',
 'Ratxa màxima del vent a 2 m': '56',
 'Ratxa màxima del vent a 6 m': '53',
 'Temperatura': '32',
 'Temperatura màxima': '40',
 'Temperatura mínima': '42',
 'Velocitat del vent a 10 m (esc.)': '30',
 'Velocitat del vent a 2 m (esc.)': '46',
 'Velocitat del vent a 6 m (esc.)': '48'}

In [ ]:
#Let's sort the dictionary
sorted_dict = sorted(dict.items(), key=operator.itemgetter(1), reverse=False)
sorted_dict

[('Pressió atmosfèrica màxima', '1'),
 ('Pressió atmosfèrica mínima', '2'),
 ('Humitat relativa màxima', '3'),
 ('Velocitat del vent a 10 m (esc.)', '30'),
 ('Direcció de vent 10 m (m. 1)', '31'),
 ('Temperatura', '32'),
 ('Humitat relativa', '33'),
 ('Pressió atmosfèrica', '34'),
 ('Precipitació', '35'),
 ('Irradiància solar global', '36'),
 ('Gruix de neu a terra', '38'),
 ('Temperatura màxima', '40'),
 ('Temperatura mínima', '42'),
 ('Humitat relativa mínima', '44'),
 ('Velocitat del vent a 2 m (esc.)', '46'),
 ('Direcció del vent a 2 m (m. 1)', '47'),
 ('Velocitat del vent a 6 m (esc.)', '48'),
 ('Direcció del vent a 6 m (m. 1)', '49'),
 ('Ratxa màxima del vent a 10 m', '50'),
 ('Direcció de la ratxa màxima del vent a 10 m', '51'),
 ('Ratxa màxima del vent a 6 m', '53'),
 ('Direcció de la ratxa màxima del vent a 6 m', '54'),
 ('Ratxa màxima del vent a 2 m', '56'),
 ('Direcció de la ratxa màxima del vent a 2 m', '57'),
 ('Irradiància neta', '59'),
 ('Precipitació màxima en 1 minut',

In [ ]:
#Let's check columns in our tableset
table2.columns

Index(['data_lectura', '1', '2', '3', '30', '31', '32', '33', '34', '35', '36',
       '40', '42', '44', '50', '51', '72'],
      dtype='object')

In [ ]:
#Rename columns with feature labels
table2.columns = ['data_lectura','Pressió atmosfèrica màxima',
 'Pressió atmosfèrica mínima',
 'Humitat relativa màxima',
 'Velocitat del vent a 10 m (esc.)',
 'Direcció de vent 10 m (m. 1)',
 'Temperatura',
 'Humitat relativa',
 'Pressió atmosfèrica',
 'Precipitació',
 'Irradiància solar global',
 'Temperatura màxima',
 'Temperatura mínima',
 'Humitat relativa mínima',
 'Ratxa màxima del vent a 10 m',
 'Direcció de la ratxa màxima del vent a 10 m',
 'Precipitació màxima en 1 minut']

In [ ]:
#Check dataframe results
table2

,data_lectura,Pressió atmosfèrica màxima,Pressió atmosfèrica mínima,Humitat relativa màxima,Velocitat del vent a 10 m (esc.),Direcció de vent 10 m (m. 1),Temperatura,Humitat relativa,Pressió atmosfèrica,Precipitació,Irradiància solar global,Temperatura màxima,Temperatura mínima,Humitat relativa mínima,Ratxa màxima del vent a 10 m,Direcció de la ratxa màxima del vent a 10 m,Precipitació màxima en 1 minut
0,2010-01-01,1003.531915,1002.680851,45.191489,5.848936,267.234043,11.278723,44.170213,1003.127660,0.000000,71.574468,11.444681,11.102128,43.042553,12.021277,271.957447,0.000000
1,2010-01-02,1016.875000,1016.312500,57.020833,3.537500,220.875000,9.833333,55.625000,1016.645833,0.000000,61.458333,10.056250,9.604167,54.312500,6.714583,216.187500,0.000000
2,2010-01-03,1016.458333,1016.125000,79.250000,1.527083,101.083333,9.666667,78.000000,1016.312500,0.000000,93.895833,9.927083,9.418750,76.687500,2.870833,118.208333,0.000000
3,2010-01-04,1008.625000,1008.354167,82.583333,1.802083,81.458333,9.406250,81.625000,1008.500000,0.250000,36.604167,9.572917,9.233333,80.500000,3.570833,82.333333,0.027083
4,2010-01-05,1000.083333,999.833333,86.187500,1.868750,158.625000,9.716667,85.458333,999.979167,0.079167,14.375000,9.787500,9.654167,84.625000,3.497917,174.458333,0.022917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4010,2020-12-27,1009.404167,1008.766667,65.937500,1.931250,217.395833,6.831250,63.083333,1009.087500,0.000000,59.062500,7.322917,6.295833,60.708333,4.272917,240.083333,0.000000
4011,2020-12-28,995.566667,994.956250,49.062500,5.014583,265.958333,10.668750,47.375000,995.262500,0.000000,106.604167,10.983333,10.360417,45.750000,10.270833,269.041667,0.000000
4012,2020-12-29,1001.879167,1001.518750,47.687500,3.875000,258.208333,10.533333,46.104167,1001.689583,0.000000,91.479167,10.879167,10.147917,44.708333,8.314583,270.229167,0.000000
4013,2020-12-30,1010.639583,1010.081250,49.250000,6.022917,295.020833,9.764583,48.312500,1010.368750,0.000000,77.208333,9.947917,9.558333,47.354167,12.089583,293.333333,0.000000


In [ ]:
#Confirm features type
table2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4015 entries, 0 to 4014
Data columns (total 17 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   data_lectura                                 4015 non-null   object 
 1   Pressió atmosfèrica màxima                   4014 non-null   float64
 2   Pressió atmosfèrica mínima                   4014 non-null   float64
 3   Humitat relativa màxima                      4014 non-null   float64
 4   Velocitat del vent a 10 m (esc.)             4015 non-null   float64
 5   Direcció de vent 10 m (m. 1)                 4014 non-null   float64
 6   Temperatura                                  4013 non-null   float64
 7   Humitat relativa                             4014 non-null   float64
 8   Pressió atmosfèrica                          4014 non-null   float64
 9   Precipitació                                 4015 non-null   float64
 10  

# 5. Export Dataset

In [ ]:
#Finally, Meteorological dataset is exported to csv to be merged with Polluntants
table2.to_csv(r'Data_sets/Meteo_ready.csv')